# Data Cleaning and Consolidation

Currently, the data are separated in daily csv files that could have different formats. Our goal in this notebook is to consolidate the daily csvs into a single csv file.

In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
CSV_PATH = os.path.join('data', 'enhanced_sur')
csv_filenames = {}

for file in os.listdir(CSV_PATH):
    if file.endswith('.csv'):
        csv_filenames[file[:8]] = file

print('Found {} CSV Files'.format(len(csv_filenames)))

Found 163 CSV Files


In [3]:
csvs = []

for key in csv_filenames:
    csvs.append(pd.read_csv(os.path.join(CSV_PATH, csv_filenames[key])))
    csvs[-1].month = key[4:6]
    csvs[-1].day = key[6:]

## Data Cleaning

See if the columns have changed

In [4]:
last_columns = None

for csv in csvs:
    if not all(last_columns == csv.columns):
        print(csv.month, csv.day, csv.columns[csv.columns != last_columns])
        last_columns = csv.columns

04 04 Index(['Case no.', 'Report date', 'Date of onset ', 'Gender', 'Age',
       'Name of hospital admitted', 'Hospitalised/Discharged/Deceased',
       'HK/Non-HK resident', 'Case classification*', 'Confirmed/probable'],
      dtype='object')
04 07 Index(['Date of onset'], dtype='object')
04 14 Index(['Date of onset '], dtype='object')
04 21 Index(['Date of onset'], dtype='object')
04 28 Index(['Date of onset '], dtype='object')
05 05 Index(['Date of onset'], dtype='object')
05 12 Index(['Date of onset '], dtype='object')
05 19 Index(['Date of onset'], dtype='object')
05 27 Index(['Date of onset '], dtype='object')
06 02 Index(['Date of onset'], dtype='object')
06 09 Index(['Date of onset '], dtype='object')
06 16 Index(['Date of onset'], dtype='object')
06 23 Index(['Date of onset '], dtype='object')
06 30 Index(['Date of onset'], dtype='object')
07 07 Index(['Date of onset '], dtype='object')
07 14 Index(['Date of onset'], dtype='object')
07 21 Index(['Date of onset '], dtype='obje

Interestingly, the column for date of onset alternates between having a trailing space and not each week. So, all we need to change is the column name.

In [5]:
for csv in csvs:
    try:
        csv.rename(columns=(lambda col: col.strip()), inplace=True)
    except:
        print(csv.month, csv.day)

### Report date

In [6]:
for csv in csvs:
    try:
        csv['Report date'] = pd.to_datetime(csv['Report date'], dayfirst=True)
    except:
        print(csv.month, csv.day)

In [7]:
for csv in csvs:
    csv['Asymptomatic'] = csv['Date of onset'] == 'Asymptomatic'

In [8]:
def dates_map(date):
    date = date.title()
    if 'Jan' in date:
        date = '01/01/2020'
    elif 'Feb' in date:
        date = '01/02/2020'
    elif 'Mar' in date:
        date = '01/03/2020'
    elif 'Apr' in date:
        date = '01/04/2020'
    elif 'May' in date:
        date = '01/05/2020'
    elif 'Jun' in date:
        date = '01/06/2020'
    elif 'Jul' in date:
        date = '01/07/2020'
    elif 'Aug' in date:
        date = '01/08/2020'
    elif 'Sep' in date:
        date = '01/09/2020'
    elif 'Oct' in date:
        date = '01/10/2020'
    elif 'Nov' in date:
        date = '01/11/2020'
    elif 'Dec' in date:
        date = '01/12/2020'
    return date

for csv in csvs:
    csv['Date of onset'] = csv['Date of onset'].map(dates_map)
    csv['Date of onset'] = pd.to_datetime(csv['Date of onset'], errors='coerce', dayfirst=True)

Now, let's check the categorical values

### Gender

In [9]:
all_genders = None

for csv in csvs:
    csv['Gender'] = csv['Gender'].map(lambda ele: ele.title())
    if all_genders is None:
        all_genders = csv['Gender']
    else:
        all_genders = all_genders.append(csv['Gender'], ignore_index=True)

all_genders.value_counts()

M          184262
F          175886
Pending         1
Name: Gender, dtype: int64

In [10]:
def gender_map(gender):
    if gender == 'Pending':
        return np.nan
    else:
        return gender

csv['Gender'] = csv['Gender'].map(gender_map)

### Name of Hospital Admitted

In [11]:
all_hospitals = None

for csv in csvs:
    csv['Name of hospital admitted'] = csv['Name of hospital admitted'].astype(np.str).map(lambda ele: ele.title())
    if all_hospitals is None:
        all_hospitals = csv['Name of hospital admitted']
    else:
        all_hospitals = all_hospitals.append(csv['Name of hospital admitted'], ignore_index=True)

all_hospitals.value_counts()

Nan                                         241229
Princess Margaret Hospital                   16194
Queen Elizabeth Hospital                     13658
Pamela Youde Nethersole Eastern Hospital     13639
United Christian Hospital                    13611
Queen Mary Hospital                          12962
Tuen Mun Hospital                            12470
Prince Of Wales Hospital                     11361
Ruttonjee Hospital                            6419
Alice Ho Miu Ling Nethersole Hospital         4768
North District Hospital                       3885
Tseung Kwan O Hospital                        3020
Caritas Medical Centre                        2403
Kwong Wah Hospital                            2069
Yan Chai Hospital                             1794
Pok Oi Hospital                                210
North Lantau Hospital                          210
Pending                                        192
Not Applicable                                  55
Name: Name of hospital admitted

In [12]:
def hospitals_map(hospital):
    if (hospital == 'Pending' or hospital == 'Not Applicable' or 
        hospital == 'Nan'):
        return np.nan
    else:
        return hospital

for csv in csvs:
    csv['Name of hospital admitted'] = csv['Name of hospital admitted'].map(hospitals_map)

In [13]:
all_hospitals = None

for csv in csvs:
    if all_hospitals is None:
        all_hospitals = csv['Name of hospital admitted']
    else:
        all_hospitals = all_hospitals.append(csv['Name of hospital admitted'], ignore_index=True)

all_hospitals.value_counts()

Princess Margaret Hospital                  16194
Queen Elizabeth Hospital                    13658
Pamela Youde Nethersole Eastern Hospital    13639
United Christian Hospital                   13611
Queen Mary Hospital                         12962
Tuen Mun Hospital                           12470
Prince Of Wales Hospital                    11361
Ruttonjee Hospital                           6419
Alice Ho Miu Ling Nethersole Hospital        4768
North District Hospital                      3885
Tseung Kwan O Hospital                       3020
Caritas Medical Centre                       2403
Kwong Wah Hospital                           2069
Yan Chai Hospital                            1794
North Lantau Hospital                         210
Pok Oi Hospital                               210
Name: Name of hospital admitted, dtype: int64

### Hospitalised/Discharged/Deceased

In [14]:
all_status = None

for csv in csvs:
    csv['Hospitalised/Discharged/Deceased'] = csv['Hospitalised/Discharged/Deceased'].astype(np.str).map(lambda ele: ele.title())
    if all_status is None:
        all_status = csv['Hospitalised/Discharged/Deceased']
    else:
        all_status = all_status.append(csv['Hospitalised/Discharged/Deceased'], ignore_index=True)

all_status.value_counts()

Discharged                                   291260
Hospitalised                                  58291
Deceased                                       4307
To Be Provided                                 2865
Pending Admission                              1811
No Admission                                   1366
Nan                                             181
Discharged (Readmitted On 21/4)                  31
Discharged (Readmitted On 18/4)                  11
Discharged (Readmitted On 24/4)                  10
No Admission As Departed                          9
Discharged (Readmitted On 20/4)                   5
Discharged (Readmitted In Mainland China)         1
Pending                                           1
Name: Hospitalised/Discharged/Deceased, dtype: int64

In [15]:
def status_map(status):
    if status.startswith('Discharged'):
        return 'Discharged'
    elif (status == 'Pending Admission' or status == 'To Be Provided' or
          status == 'Nan' or status == 'Pending'):
        return np.nan
    elif (status.startswith('No')):
        return 'No Admission'
    else:
        return status

for csv in csvs:
    csv['Hospitalised/Discharged/Deceased'] = csv['Hospitalised/Discharged/Deceased'].map(status_map)

In [16]:
all_status = None

for csv in csvs:
    if all_status is None:
        all_status = csv['Hospitalised/Discharged/Deceased']
    else:
        all_status = all_status.append(csv['Hospitalised/Discharged/Deceased'], ignore_index=True)

all_status.value_counts()

Discharged      291318
Hospitalised     58291
Deceased          4307
No Admission      1375
Name: Hospitalised/Discharged/Deceased, dtype: int64

### HK/Non-HK resident

In [17]:
all_residence = None

for csv in csvs:
    csv['HK/Non-HK resident'] = csv['HK/Non-HK resident'].astype(np.str).map(lambda ele: ele.title())
    if all_residence is None:
        all_residence = csv['HK/Non-HK resident']
    else:
        all_residence = all_residence.append(csv['HK/Non-HK resident'], ignore_index=True)

all_residence.value_counts()

Hk Resident        350886
Unknown              5893
Non-Hk Resident      3369
Pending                 1
Name: HK/Non-HK resident, dtype: int64

In [18]:
def residence_map(resident):
    if resident == 'Unknown' or resident == 'Pending':
        return np.nan
    elif resident == 'Hk Resident':
        return True
    elif resident == 'Non-Hk Resident':
        return False
    else:
        return resident

for csv in csvs:
    csv['HK/Non-HK resident'] = csv['HK/Non-HK resident'].map(residence_map)

In [19]:
all_residence = None

for csv in csvs:
    if all_residence is None:
        all_residence = csv['HK/Non-HK resident']
    else:
        all_residence = all_residence.append(csv['HK/Non-HK resident'], ignore_index=True)

all_residence.value_counts()

True     350886
False      3369
Name: HK/Non-HK resident, dtype: int64

### Case Classification

In [20]:
all_classification = None

for csv in csvs:
    csv['Case classification*'] = csv['Case classification*'].astype(np.str).map(lambda ele: ele.title())
    if all_classification is None:
        all_classification = csv['Case classification*']
    else:
        all_classification = all_classification.append(csv['Case classification*'], ignore_index=True)

all_classification.value_counts()

Imported Case                                        118602
Epidemiologically Linked With Local Case             116948
Local Case                                            71838
Imported                                              16944
Possibly Local Case                                   13699
Epidemiologically Linked With Possibly Local Case      8432
Close Contact Of Local Case                            4630
Epidemiologically Linked With Imported Case            3855
Possibly Local                                         3080
Close Contact Of Possibly Local Case                   1342
Close Contact Of Imported Case                          613
Epidemiologically Linked With Local Case)               165
Pending                                                   1
Name: Case classification*, dtype: int64

In [21]:
def classification_map(clf):
    if clf.startswith('Imported'):
        return 'Imported'
    elif clf == 'Local Case' or clf == 'Possibly Local Case' or clf == 'Possibly Local':
        return 'Local'
    elif ('Epidemiologically' in clf) and ('Local' in clf):
        return 'EL-L'
    elif ('Epidemiologically' in clf) and ('Imported' in clf):
        return 'EL-I'
    elif ('Contact' in clf) and ('Local' in clf):
        return 'CC-L'
    elif ('Contact' in clf) and ('Import' in clf):
        return 'CC-I'
    elif clf == 'Pending' or clf == 'Nan':
        return np.nan
    else:
        return clf
    
for csv in csvs:
    csv['Case classification*'] = csv['Case classification*'].map(classification_map)

In [22]:
all_classification = None

for csv in csvs:
    if all_classification is None:
        all_classification = csv['Case classification*']
    else:
        all_classification = all_classification.append(csv['Case classification*'], ignore_index=True)

all_classification.value_counts()

Imported    135546
EL-L        125545
Local        88617
CC-L          5972
EL-I          3855
CC-I           613
Name: Case classification*, dtype: int64

### Confirmed/probable

In [23]:
all_confirmed = None

for csv in csvs:
    csv['Confirmed/probable'] = csv['Confirmed/probable'].astype(np.str).map(lambda ele: ele.title())
    if all_confirmed is None:
        all_confirmed = csv['Confirmed/probable']
    else:
        all_confirmed = all_confirmed.append(csv['Confirmed/probable'], ignore_index=True)

all_confirmed.value_counts()

Confirmed    359986
Probable        163
Name: Confirmed/probable, dtype: int64

In [24]:
for csv in csvs:
    csv['Confirmed/probable'] = csv['Confirmed/probable'].map(lambda conf: conf == 'Confirmed')

### Rename Columns

In [25]:
for csv in csvs:
    csv.rename(columns = {
        'Report date': 'Reported',
        'Date of onset': 'Onset',
        'Name of hospital admitted': 'Admitted Hospital',
        'Hospitalised/Discharged/Deceased': 'Status',
        'HK/Non-HK resident': 'Resident',
        'Case classification*': 'Case Type',
        'Confirmed/probable': 'Confirmed',
    }, inplace=True)

### Preview Cleaned Results

In [26]:
import ipywidgets as widgets

def preview_df(df):
    df = max(df, 0)
    df = min(df, len(csvs)-1)
    return csvs[df].iloc[np.random.randint(len(csvs[df]), size=20)]

widgets.interact(preview_df, df=(0, len(csvs)-1));

interactive(children=(IntSlider(value=81, description='df', max=162), Output()), _dom_classes=('widget-interac…

## Consolidate Tables into a Progress DF

With rows being case ID, and columns being the different statuses. The value will be dates the status first appeared.

In [27]:
progress = pd.DataFrame(columns=['Gender', 'Age', 'Resident', 'Case Type', 
                                 'Reported', 'Onset', 'Asymptomatic', 
                                 'Hospitalised', 'Discharged', 'Deceased', 
                                 'Confirmed'])
progress.index.name = 'Case no.'

def col_map(row, match, day, month):
    if row == match:
        return pd.to_datetime(day + '/' + month + '/2020', dayfirst=True)
    else:
        return np.nan

# Start from most recent so earlier dates will overwrite later dates
for i in range(len(csvs)-1, -1, -1):
    csv = csvs[i]
    print('Adding data from {}/{}/2020'.format(csv.day, csv.month))
    progress = pd.merge(progress, 
                        csv['Case no.'], 
                        on=['Case no.'], 
                        how='outer')
    progress.update(csv['Gender'])
    progress.update(csv['Age'])
    progress.update(csv['Resident'])
    progress.update(csv['Case Type'])
    progress.update(csv['Asymptomatic'])
    progress.update(csv['Reported'])
    progress.update(csv['Onset'])
    hospitalised = csv['Status'].map(lambda row: col_map(row, 'Hospitalised',
                                                         csv.day, csv.month))
    hospitalised.name = 'Hospitalised'
    progress.update(hospitalised)
    discharged = csv['Status'].map(lambda row: col_map(row, 'Discharged',
                                                         csv.day, csv.month))
    discharged.name = 'Discharged'
    progress.update(discharged)
    deceased = csv['Status'].map(lambda row: col_map(row, 'Deceased',
                                                         csv.day, csv.month))
    deceased.name = 'Deceased'
    progress.update(deceased)
    confirmed = csv['Confirmed'].map(lambda row: col_map(row, True,
                                                         csv.day, csv.month))
    confirmed.name = 'Confirmed'
    progress.update(confirmed)
    
for date_col in ['Reported', 'Onset', 'Hospitalised', 'Discharged', 
                 'Deceased', 'Confirmed']:
    progress[date_col] = progress[date_col].astype(np.datetime64)

Adding data from 18/09/2020
Adding data from 17/09/2020
Adding data from 16/09/2020
Adding data from 15/09/2020
Adding data from 14/09/2020
Adding data from 13/09/2020
Adding data from 12/09/2020
Adding data from 11/09/2020
Adding data from 10/09/2020
Adding data from 09/09/2020
Adding data from 08/09/2020
Adding data from 07/09/2020
Adding data from 06/09/2020
Adding data from 05/09/2020
Adding data from 04/09/2020
Adding data from 03/09/2020
Adding data from 02/09/2020
Adding data from 01/09/2020
Adding data from 31/08/2020
Adding data from 30/08/2020
Adding data from 29/08/2020
Adding data from 28/08/2020
Adding data from 27/08/2020
Adding data from 26/08/2020
Adding data from 25/08/2020
Adding data from 24/08/2020
Adding data from 23/08/2020
Adding data from 22/08/2020
Adding data from 21/08/2020
Adding data from 20/08/2020
Adding data from 19/08/2020
Adding data from 18/08/2020
Adding data from 17/08/2020
Adding data from 16/08/2020
Adding data from 15/08/2020
Adding data from 14/

In [28]:
progress.set_index(progress['Case no.'], drop=True, inplace=True)
progress.drop(columns=['Case no.'], inplace=True)

In [29]:
def preview_progress(row, rows=20):
    row = max(0, row)
    row = min(len(progress)-rows, row)
    return progress.iloc[row:row+rows]

widgets.interact(preview_progress, row=(0, len(progress) - 1));

interactive(children=(IntSlider(value=2496, description='row', max=4993), IntSlider(value=20, description='row…

In [30]:
progress.to_csv('data\\cleaned_progress.csv')